In [623]:
from numpy.random import seed
import pandas as pd
import numpy as np
from sklearn.preprocessing import minmax_scale
# from keras_diagram import ascii
from keras.layers.convolutional import Conv2D, MaxPooling2D, Conv1D, MaxPooling1D
from keras.optimizers import SGD
from keras.models import Sequential
from keras.layers import Dense, Flatten
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_recall_fscore_support
import keras
import math
data_raw = pd.read_csv("creditcard.csv")



In [624]:
# Assign variables x and y corresponding to row data and it's class value
X = data_raw.loc[:, data.columns != 'Class']
y = data_raw.loc[:, data.columns == 'Class']

In [625]:
int(X.shape[0]*0.7)

199364

In [626]:
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

def generate_train_test_timeseries(data, test_ratio=0.3):
#     cutoff = math.floor(X.shape[0] *cuttoff_percent) 
#     print cutoff
    total_samples = data.shape[0]
    # splitting index
    cutoff = int(total_samples * (1 - test_ratio))

    
    data.sort_values('Time', inplace=True)
    
    X_train = data.loc[0:cutoff, data.columns != 'Class']
    y_train = data.loc[0:cutoff, data.columns == 'Class']
    X_test = data.loc[cutoff:, data.columns != 'Class']
    y_test = data.loc[cutoff:, data.columns == 'Class']
    
    pipeline = Pipeline([
        ('scaling', StandardScaler()),
    ])
    preprocessor = pipeline.fit(X_train)
    X_train_prp = preprocessor.transform(X_train)
    
#     preprocessor = pipeline.fit(X_test)
    X_test_prp = preprocessor.transform(X_test)
    
#     # Normalise ALL columns - X_Train
#     from sklearn.preprocessing import StandardScaler
#     norm_cols = StandardScaler().fit_transform(X_train)
#     X_train = pd.DataFrame(norm_cols)
    
#     # Normalise ALL columns - X_Test
#     from sklearn.preprocessing import StandardScaler
#     norm_cols = StandardScaler().fit_transform(X_test)
#     X_test = pd.DataFrame(norm_cols)
    
    
    return X_train_prp, y_train, X_test_prp, y_test
    
########################################################################

# X_res, y_res, X_test, y_test = generate_train_test_sample(X, y)

# print X_res.shape, type(X_res)
# print y_res.shape

# X_train = X_res.reshape(X_res.shape[0], 29, 1)
# Y_train = y_res.reshape(y_res.shape[0], 1)
# X_test = X_test.values.reshape(X_test.values.shape[0], 29, 1)
# Y_test = y_test.values.reshape(y_test.values.shape[0], 1)

# Y_test = keras.utils.to_categorical(Y_test)
# Y_train = keras.utils.to_categorical(Y_train)
# print Y_test.shape
# print Y_train.shape




In [627]:
X_train, y_train, X_test, y_test = generate_train_test_timeseries(data_raw)

# observe the mean and the variance of the tranformed data set for each fature
checker = lambda x: [np.mean(x), np.var(x)]
pd.DataFrame(X_train).apply(checker)

0      [2.37221801681e-16, 1.0]
1      [1.65370967518e-17, 1.0]
2     [-3.13634593569e-18, 1.0]
3     [-4.79005561087e-17, 1.0]
4      [1.71073414674e-18, 1.0]
5       [2.1669299192e-17, 1.0]
6      [4.27683536685e-18, 1.0]
7     [-2.28097886232e-18, 1.0]
8     [-1.42561178895e-18, 1.0]
9      [2.62312569167e-17, 1.0]
10    [-4.27683536685e-18, 1.0]
11     [5.93054504203e-17, 1.0]
12     [3.19337040725e-17, 1.0]
13     [1.73924638252e-17, 1.0]
14     [-2.8512235779e-17, 1.0]
15     [1.25453837428e-17, 1.0]
16    [-1.99585650453e-18, 1.0]
17     [1.14048943116e-18, 1.0]
18     [-1.0834649596e-17, 1.0]
19     [1.26166643322e-17, 1.0]
20    [-1.14048943116e-18, 1.0]
21    [-3.99171300906e-18, 1.0]
22     [3.42146829348e-17, 1.0]
23     [5.98756951359e-18, 1.0]
24     [2.49482063066e-17, 1.0]
25    [-6.95698553008e-17, 1.0]
26    [-9.26647662818e-18, 1.0]
27    [-8.26854837591e-18, 1.0]
28    [-9.12391544928e-18, 1.0]
29      [4.2055547774e-18, 1.0]
dtype: object

In [628]:
pd.DataFrame(X_test).apply(checker)

0         [2.3542902492, 0.101162296229]
1        [0.185810693741, 1.21860638875]
2     [-0.00771279211375, 1.12244913422]
3      [-0.738688825683, 0.942061396417]
4        [-0.17983654767, 1.05712304367]
5        [0.274557741571, 1.03950113753]
6       [-0.116894042519, 1.09946461167]
7        [0.138438463223, 1.09153568284]
8     [-0.0472197909783, 0.914012666523]
9     [-0.0258953529381, 0.785262337184]
10     [0.0342903908915, 0.975745745013]
11     [-0.403980218415, 0.757623382476]
12      [0.213213479705, 0.415170985072]
13      [-0.12035072016, 0.757388461736]
14      [-0.174103500009, 0.88856693772]
15         [-0.297065407, 0.73485281857]
16     [0.0223151840834, 0.927390725625]
17    [-0.0997550510562, 0.690374296232]
18      [0.143057666019, 0.974411197554]
19     [0.0543747049487, 0.956058532395]
20     [-0.0884062249473, 1.19046798698]
21     [0.0662583401281, 0.963964937722]
22         [0.231040162274, 1.302401999]
23      [0.0846970360626, 1.14626364524]
24     [-0.02366

In [629]:
X_train.shape

(199365, 30)

In [630]:
data_raw[data_raw['Class']==1]

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
541,406.0,-2.312227,1.951992,-1.609851,3.997906,-0.522188,-1.426545,-2.537387,1.391657,-2.770089,...,0.517232,-0.035049,-0.465211,0.320198,0.044519,0.177840,0.261145,-0.143276,0.00,1
623,472.0,-3.043541,-3.157307,1.088463,2.288644,1.359805,-1.064823,0.325574,-0.067794,-0.270953,...,0.661696,0.435477,1.375966,-0.293803,0.279798,-0.145362,-0.252773,0.035764,529.00,1
4920,4462.0,-2.303350,1.759247,-0.359745,2.330243,-0.821628,-0.075788,0.562320,-0.399147,-0.238253,...,-0.294166,-0.932391,0.172726,-0.087330,-0.156114,-0.542628,0.039566,-0.153029,239.93,1
6108,6986.0,-4.397974,1.358367,-2.592844,2.679787,-1.128131,-1.706536,-3.496197,-0.248778,-0.247768,...,0.573574,0.176968,-0.436207,-0.053502,0.252405,-0.657488,-0.827136,0.849573,59.00,1
6329,7519.0,1.234235,3.019740,-4.304597,4.732795,3.624201,-1.357746,1.713445,-0.496358,-1.282858,...,-0.379068,-0.704181,-0.656805,-1.632653,1.488901,0.566797,-0.010016,0.146793,1.00,1
6331,7526.0,0.008430,4.137837,-6.240697,6.675732,0.768307,-3.353060,-1.631735,0.154612,-2.795892,...,0.364514,-0.608057,-0.539528,0.128940,1.488481,0.507963,0.735822,0.513574,1.00,1
6334,7535.0,0.026779,4.132464,-6.560600,6.348557,1.329666,-2.513479,-1.689102,0.303253,-3.139409,...,0.370509,-0.576752,-0.669605,-0.759908,1.605056,0.540675,0.737040,0.496699,1.00,1
6336,7543.0,0.329594,3.712889,-5.775935,6.078266,1.667359,-2.420168,-0.812891,0.133080,-2.214311,...,0.156617,-0.652450,-0.551572,-0.716522,1.415717,0.555265,0.530507,0.404474,1.00,1
6338,7551.0,0.316459,3.809076,-5.615159,6.047445,1.554026,-2.651353,-0.746579,0.055586,-2.678679,...,0.208828,-0.511747,-0.583813,-0.219845,1.474753,0.491192,0.518868,0.402528,1.00,1
6427,7610.0,0.725646,2.300894,-5.329976,4.007683,-1.730411,-1.732193,-3.968593,1.063728,-0.486097,...,0.589669,0.109541,0.601045,-0.364700,-1.843078,0.351909,0.594550,0.099372,1.00,1


In [631]:
import math


X_train, y_train, X_test, y_test = generate_train_test_timeseries(data_raw)
print X_train.shape
print y_train.shape



(199365, 30)
(199365, 1)


In [632]:
X_train

array([[-2.04169282, -0.66333395, -0.04718907, ...,  0.33491107,
        -0.07318358,  0.24044257],
       [-2.04169282,  0.68601299,  0.16179919, ..., -0.02682455,
         0.04081383, -0.34985017],
       [-2.04166384, -0.66256555, -0.82866747, ..., -0.14449956,
        -0.19649   ,  1.16061305],
       ..., 
       [ 1.81094662, -0.67591045, -3.47526897, ..., -1.05173088,
         0.61955003,  6.65150461],
       [ 1.81094662, -0.78522762,  0.72888384, ..., -0.28086169,
        -0.40880546, -0.19879179],
       [ 1.81097561, -0.73988366, -0.64800196, ..., -0.33681341,
        -0.41196738,  0.11573906]])

In [633]:
print X_train.shape
print y_train.shape
print X_test.shape
print y_test.shape

(199365, 30)
(199365, 1)
(85443, 30)
(85443, 1)


In [634]:
#com add your code here
import seaborn as sns
#hide
sns.distplot(X_train[:, 0])
#/hide

In [635]:
def reshape_to_batches(a, batch_size):
    # pad with zeros if the length is not divisible by the batch_size
    batch_num = np.ceil((float)(a.shape[0]) / batch_size)
    modulo = batch_num * batch_size - a.shape[0]
    if modulo != 0:
        pad = np.zeros((int(modulo), a.shape[1]))
        a = np.vstack((a, pad))
    return np.array(np.split(a, batch_num))

In [636]:
X_train_batch = reshape_to_batches(X_train, 100)

In [637]:
X_train_batch.shape

(1994, 100, 30)

In [638]:
y_test.shape

(85443, 1)

In [639]:
y_test = keras.utils.to_categorical(y_test)
y_train = keras.utils.to_categorical(y_train)

In [640]:
y_test.shape

(85443, 2)

In [641]:
y_test

array([[ 1.,  0.],
       [ 1.,  0.],
       [ 1.,  0.],
       ..., 
       [ 1.,  0.],
       [ 1.,  0.],
       [ 1.,  0.]])

In [642]:
y_train_batch = reshape_to_batches(y_train, 100)
y_test_batch = reshape_to_batches(y_test, 100)
X_test_batch = reshape_to_batches(X_test, 100)

In [643]:
print X_train_batch.shape
print X_test_batch.shape
print y_train_batch.shape
print y_test_batch.shape

(1994, 100, 30)
(855, 100, 30)
(1994, 100, 2)
(855, 100, 2)


In [644]:
#import all dependencies
from keras.layers import Input, Dense, Conv1D
from keras.models import Model

seed(2018)

In [645]:
inputs = Input(shape=(100, 30)) # This returns a tensor

In [646]:
# a layer instance is callable on a tensor, and returns a tensor
conv1 = Conv1D(32, (5), # 32 filters with a window of width 5
strides=1, # think autoregression
padding='causal', # forward in time
dilation_rate=1, # ignore this and everything that follows are default parameters
activation='relu',
use_bias=True,
kernel_initializer='glorot_uniform',
bias_initializer='zeros',
kernel_regularizer=None, # no regularisation for the moment
bias_regularizer=None,
activity_regularizer=None,
kernel_constraint=None,
bias_constraint=None)(inputs) # syntax to chain layers: Layer(...)(PreviousLayer)

In [647]:
fc1 = Dense(64, activation='relu')(conv1)

In [648]:
predictions = Dense(2, activation='softmax')(fc1)

In [649]:
model = Model(inputs=inputs,
outputs=predictions)
model.compile(optimizer='rmsprop',
loss='categorical_crossentropy',
metrics=['accuracy'])

In [650]:
X_train_batch

array([[[-2.04169282, -0.66333395, -0.04718907, ...,  0.33491107,
         -0.07318358,  0.24044257],
        [-2.04169282,  0.68601299,  0.16179919, ..., -0.02682455,
          0.04081383, -0.34985017],
        [-2.04166384, -0.66256555, -0.82866747, ..., -0.14449956,
         -0.19649   ,  1.16061305],
        ..., 
        [-2.03975097, -0.28980201,  0.09648455, ...,  0.02449283,
         -0.16375542, -0.2809097 ],
        [-2.03975097, -0.73464971,  0.51393854, ..., -0.80972917,
         -1.09194432, -0.24704209],
        [-2.03972199,  0.70776792,  0.11450719, ..., -0.08812375,
          0.06077998, -0.29641728]],

       [[-2.03972199,  0.66754796,  0.02063557, ..., -0.07207959,
          0.00727618, -0.34985017],
        [-2.03972199, -0.35154157,  0.29586326, ...,  0.14564766,
          0.42146265, -0.27082575],
        [-2.039693  ,  0.21359801,  1.31890862, ...,  0.63331734,
         -0.77807727, -0.35752361],
        ..., 
        [-2.03792505,  0.77137675, -0.43079279, ...,

In [651]:
model.fit(X_train_batch, y_train_batch, epochs=30)

Epoch 1/30
1994/1994 [==============================] - 2s 763us/step - loss: 0.0973 - acc: 0.9869
Epoch 2/30
1994/1994 [==============================] - 1s 273us/step - loss: 0.0067 - acc: 0.9987
Epoch 3/30
1994/1994 [==============================] - 1s 262us/step - loss: 0.0040 - acc: 0.9992
Epoch 4/30
1994/1994 [==============================] - 1s 263us/step - loss: 0.0033 - acc: 0.9993
Epoch 5/30
1994/1994 [==============================] - 1s 262us/step - loss: 0.0029 - acc: 0.9994
Epoch 6/30
1994/1994 [==============================] - 1s 265us/step - loss: 0.0025 - acc: 0.9994
Epoch 7/30
1994/1994 [==============================] - 1s 261us/step - loss: 0.0022 - acc: 0.9994
Epoch 8/30
1994/1994 [==============================] - 1s 265us/step - loss: 0.0020 - acc: 0.9995
Epoch 9/30
1994/1994 [==============================] - 1s 266us/step - loss: 0.0017 - acc: 0.9995
Epoch 10/30
1994/1994 [==============================] - 1s 274us/step - loss: 0.0015 - acc: 0.9995
Epoch 11/

In [652]:
y_pred = model.predict(X_test_batch)

In [653]:
y_pred_3d = y_pred

# store the raw predictions we will need them in a bit
y_hat = np.copy(y_pred)

# a function to reshape batches into the original shape
def _3d_to_2d(arr):
    return arr.reshape(arr.shape[0] * arr.shape[1], arr.shape[2])

In [654]:
from sklearn.metrics import confusion_matrix, classification_report
cutt_off_tr = 0.5
y_pred[np.where(y_pred>=cutt_off_tr)] = 1
y_pred[np.where(y_pred<cutt_off_tr)]  = 0

y_pred_2d = _3d_to_2d(y_pred)[:,1]
y_test_2d = _3d_to_2d(y_test_batch)[:,1]

print(confusion_matrix(
_3d_to_2d(y_test_batch)[:, 1],
_3d_to_2d(y_pred)[:, 1]))

prfs = precision_recall_fscore_support(y_test_2d, y_pred_2d, labels=[1])
prfs0 = precision_recall_fscore_support(y_test_2d, y_pred_2d, labels=[0])
print prfs
print "\n"
print prfs0

[[85389     3]
 [   44    64]]
(array([ 0.95522388]), array([ 0.59259259]), array([ 0.73142857]), array([108]))


(array([ 0.99948498]), array([ 0.99996487]), array([ 0.99972486]), array([85392]))


In [655]:
print(classification_report(
y_test_2d,
y_pred_2d,
target_names = ["Genuine", "Fraud"],
digits = 5))

             precision    recall  f1-score   support

    Genuine    0.99948   0.99996   0.99972     85392
      Fraud    0.95522   0.59259   0.73143       108

avg / total    0.99943   0.99945   0.99939     85500



In [656]:
from sklearn.metrics import roc_curve, auc, roc_auc_score

# short way
print(roc_auc_score(y_test_2d, _3d_to_2d(y_hat)[:, 1]))

0.943559039705


In [657]:
X_train_batch

array([[[-2.04169282, -0.66333395, -0.04718907, ...,  0.33491107,
         -0.07318358,  0.24044257],
        [-2.04169282,  0.68601299,  0.16179919, ..., -0.02682455,
          0.04081383, -0.34985017],
        [-2.04166384, -0.66256555, -0.82866747, ..., -0.14449956,
         -0.19649   ,  1.16061305],
        ..., 
        [-2.03975097, -0.28980201,  0.09648455, ...,  0.02449283,
         -0.16375542, -0.2809097 ],
        [-2.03975097, -0.73464971,  0.51393854, ..., -0.80972917,
         -1.09194432, -0.24704209],
        [-2.03972199,  0.70776792,  0.11450719, ..., -0.08812375,
          0.06077998, -0.29641728]],

       [[-2.03972199,  0.66754796,  0.02063557, ..., -0.07207959,
          0.00727618, -0.34985017],
        [-2.03972199, -0.35154157,  0.29586326, ...,  0.14564766,
          0.42146265, -0.27082575],
        [-2.039693  ,  0.21359801,  1.31890862, ...,  0.63331734,
         -0.77807727, -0.35752361],
        ..., 
        [-2.03792505,  0.77137675, -0.43079279, ...,

In [680]:
len(y_hat[np.where(y_hat >= cutt_off_tr)])

85500

In [679]:
len(np.where(y_pred[1] == 1)[0])

100

In [682]:
_3d_to_2d(y_pred)[:, 1]

array([ 0.,  0.,  0., ...,  0.,  0.,  0.], dtype=float32)

In [689]:
np.where(y_hat[:,:,1]>0.5)

(array([  5,  17,  39,  39,  43,  47,  51,  92, 132, 137, 152, 166, 213,
        227, 240, 242, 242, 242, 274, 275, 303, 305, 307, 311, 352, 352,
        352, 353, 362, 362, 377, 378, 388, 401, 401, 420, 440, 441, 443,
        443, 444, 446, 449, 483, 486, 489, 498, 504, 505, 513, 521, 525,
        525, 527, 534, 560, 561, 621, 631, 634, 634, 639, 639, 645, 690,
        775, 804]),
 array([32, 36, 62, 64, 36,  0, 39, 87, 80, 54, 98, 20, 61, 69,  2,  6, 14,
        54, 50, 13, 48, 89, 13, 10, 10, 68, 69, 41, 52, 80, 43, 29, 58, 35,
        37, 81, 29, 83, 35, 85, 84, 40, 67,  9, 31, 32,  3, 64, 99, 97, 13,
         2, 40, 60, 10, 39, 92,  9, 96, 62, 79, 10, 60, 13, 10,  0, 99]))

In [692]:
y_hat[:][5]

array([[  1.00000000e+00,   2.19847505e-18],
       [  1.00000000e+00,   4.80529846e-19],
       [  1.00000000e+00,   7.84376979e-29],
       [  1.00000000e+00,   1.22740433e-32],
       [  1.00000000e+00,   0.00000000e+00],
       [  1.00000000e+00,   4.90221346e-31],
       [  1.00000000e+00,   7.77231147e-30],
       [  1.00000000e+00,   5.72898769e-26],
       [  1.00000000e+00,   6.86874995e-24],
       [  1.00000000e+00,   1.44047465e-27],
       [  1.00000000e+00,   2.54878514e-35],
       [  1.00000000e+00,   0.00000000e+00],
       [  1.00000000e+00,   0.00000000e+00],
       [  1.00000000e+00,   2.80295370e-38],
       [  1.00000000e+00,   0.00000000e+00],
       [  1.00000000e+00,   0.00000000e+00],
       [  1.00000000e+00,   1.26569988e-32],
       [  1.00000000e+00,   6.21898321e-37],
       [  1.00000000e+00,   6.10997121e-21],
       [  1.00000000e+00,   7.72909163e-21],
       [  1.00000000e+00,   8.38820638e-31],
       [  1.00000000e+00,   9.73068978e-28],
       [  

In [658]:
conv1d = Conv1D(32, (5), input_shape=(100, 30), strides=1,padding='causal')

In [659]:
seed(2017)
conv = Sequential()
conv.add(conv1d)
conv.add(Dense(64, activation='relu'))
conv.add(Dense(2, activation='softmax'))

In [660]:
sgd = SGD(lr = 0.1, momentum = 0.9, decay = 0, nesterov = False)
conv.compile(loss = 'categorical_crossentropy', optimizer = 'rmsprop', metrics = ['accuracy'])
conv.fit(X_train_batch, y_train_batch, epochs = 30, verbose = 50)

Epoch 1/30
Epoch 2/30
Epoch 3/30
Epoch 4/30
Epoch 5/30
Epoch 6/30
Epoch 7/30
Epoch 8/30
Epoch 9/30
Epoch 10/30
Epoch 11/30
Epoch 12/30
Epoch 13/30
Epoch 14/30
Epoch 15/30
Epoch 16/30
Epoch 17/30
Epoch 18/30
Epoch 19/30
Epoch 20/30
Epoch 21/30
Epoch 22/30
Epoch 23/30
Epoch 24/30
Epoch 25/30
Epoch 26/30
Epoch 27/30
Epoch 28/30
Epoch 29/30
Epoch 30/30


In [661]:
y_pred = conv.predict(X_test_batch)

In [662]:
y_pred_3d = y_pred

# a function to reshape batches into the original shape
def _3d_to_2d(arr):
    return arr.reshape(arr.shape[0] * arr.shape[1], arr.shape[2])



In [663]:
print y_pred_2d.shape
print y_test_2d.shape

(85500,)
(85500,)


In [664]:
from sklearn.metrics import confusion_matrix
cutt_off_tr = 0.5
y_pred[np.where(y_pred>=cutt_off_tr)] = 1
y_pred[np.where(y_pred<cutt_off_tr)]  = 0

y_pred_2d = _3d_to_2d(y_pred)[:,1]
y_test_2d = _3d_to_2d(y_test_batch)[:,1]

print(confusion_matrix(
_3d_to_2d(y_test_batch)[:, 1],
_3d_to_2d(y_pred)[:, 1]))

prfs = precision_recall_fscore_support(y_test_2d, y_pred_2d, labels=[1])
prfs0 = precision_recall_fscore_support(y_test_2d, y_pred_2d, labels=[0])
print prfs
print "\n"
print prfs0

[[85384     8]
 [   43    65]]
(array([ 0.89041096]), array([ 0.60185185]), array([ 0.71823204]), array([108]))


(array([ 0.99949665]), array([ 0.99990631]), array([ 0.99970144]), array([85392]))


In [665]:
y_pred_2d

array([ 0.,  0.,  0., ...,  0.,  0.,  0.], dtype=float32)

In [666]:
from sklearn.metrics import roc_curve, auc, roc_auc_score

print(roc_auc_score(y_test_2d[:, 1],
y_pred_2d[:, 1]))

IndexError: too many indices for array

In [ ]:
from collections import Counter
Counter(y_pred_2d)
